# Studying Manchester City - Arsenal using VAEP

The idea of this notebook is to used the VAEP model trained in _Training-VAEP-model.ipynb_ and the matrix of distribution of VAEP values by events and positions to analyze the game of Manchester City - Arsenal game.

We are going to use the other 5 games of the dataset to compute a mean VAEP per minute and per player in order to compare the game with the previous ones.

Of course, once the model is built the idea is to ran

**Summary**

1. [Getting and transforming the data](#data)
2. [Features & labels](#features)
3. [Computing the VAEP values](#vaep)
4. [Using the position to specialize the VAEP](#position)
5. [Analysing a game](#analyse)
6. [Providing substitute ideas](#substitute)

In [1]:
# Import librairies
import os
import warnings
import pandas as pd
import tqdm
import json
import pickle
import numpy as np
from math import ceil

# Socceraction
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.atomic.spadl as atomicspadl
import socceraction.atomic.vaep.features as fs
import socceraction.atomic.vaep.labels as lab
import socceraction.atomic.vaep.formula as vaepformula

# Viz
import plotly.graph_objs as go

# Others
from helper_preprocessing import extract_dataframe_from_json

pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

## Getting and transforming the data <a id="data"></a>

In [2]:
game_arsenal_id = 3852832

In [3]:
SBL = StatsBombLoader(getter="local", root="Data/StatsBomb-City-VAEP/Data/")
df_games = SBL.games(competition_id=37, season_id=235)

# Using the games of Manchester City
df_games_city = df_games[(df_games["home_team_id"] == 746) & (~df_games["home_score"].isna())]

# Manchester United game is not in the data and is dropped
df_games_city = df_games_city.drop(df_games_city[df_games_city["away_team_id"] == 1475].index)

df_games_city[["game_day", "home_team_id", "away_team_id", "game_date", "home_score", "away_score"]].sort_values(by="game_date")

,game_day,home_team_id,away_team_id,game_date,home_score,away_score
124,4,746,2652,2022-10-16 15:00:00,4.0,0.0
46,6,746,966,2022-10-30 16:00:00,2.0,1.0
108,9,746,965,2022-12-04 16:00:00,3.0,1.0
21,12,746,2647,2023-01-21 13:30:00,1.0,1.0
129,1,746,968,2023-02-11 14:30:00,2.0,1.0
13,14,746,749,2023-03-05 16:00:00,3.0,1.0


In [4]:
# Load and convert match data
games_verbose = tqdm.tqdm(list(df_games_city.itertuples()), desc="Loading game data")
teams, players = [], []
actions = {}
atomic_actions = {}
for game in games_verbose:
    # load data
    teams.append(SBL.teams(game.game_id))
    players.append(SBL.players(game.game_id))
    events = SBL.events(game.game_id)
    
    # convert data
    actions = spadl.statsbomb.convert_to_actions(events, game.home_team_id)
    atomic_actions[game.game_id] = atomicspadl.convert_to_atomic(actions)

teams = pd.concat(teams).drop_duplicates(subset="team_id")
players = pd.concat(players)

Loading game data: 100%|██████████| 6/6 [00:04<00:00,  1.31it/s]


In [5]:
# Store converted SPADL Data in a h5-file
datafolder = "Data/data-spadl"

Transforming the data exactly what we did in the previous notebook for coding quicker

In [6]:
# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)
    print(f"Directory {datafolder} created.")

spadl_h5 = os.path.join(datafolder, "atomic-spadl-statsbomb.h5")

# Store all spadl data in h5-file
with pd.HDFStore(spadl_h5) as spadlstore:
    spadlstore["games"] = df_games_city
    spadlstore["teams"] = teams
    spadlstore["players"] = players[['player_id', 'player_name', 'nickname']].drop_duplicates(subset='player_id')
    spadlstore["player_games"] = players[['player_id', 'game_id', 'team_id', 'is_starter', 'starting_position_id', 'starting_position_name', 'minutes_played']]
    for game_id in atomic_actions.keys():
        spadlstore[f"atomic_actions/game_{game_id}"] = atomic_actions[game_id]

## Read the data and put it in the right format
with pd.HDFStore(spadl_h5) as spadlstore:
    games = (
        spadlstore["games"]
        .merge(spadlstore["teams"].add_prefix('home_'), how='left')
        .merge(spadlstore["teams"].add_prefix('away_'), how='left'))

In [7]:
# Choosing the Arsenal game 
with pd.HDFStore(spadl_h5) as spadlstore:
    atomic_actions_mancity_arsenal = spadlstore[f"atomic_actions/game_{game_arsenal_id}"]
    atomic_actions_mancity_arsenal = (
        atomic_actions_mancity_arsenal
        .merge(atomicspadl.actiontypes_df(), how="left")
        .merge(spadl.bodyparts_df(), how="left")
        .merge(spadlstore["players"], how="left")
        .merge(spadlstore["teams"], how="left")
    )

# use nickname if available else use full name
atomic_actions_mancity_arsenal["player_name"] = atomic_actions_mancity_arsenal[["nickname", "player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)
del atomic_actions_mancity_arsenal['nickname']

atomic_actions_mancity_arsenal.sample(3)


,game_id,original_event_id,action_id,period_id,time_seconds,team_id,player_id,x,y,dx,dy,type_id,bodypart_id,type_name,bodypart_name,player_name,team_name
653,3852832,fe504655-499e-43c2-a081-2657acfa125a,653,1,970.0,746,25554.0,35.117647,22.982278,-4.500000,4.992405,0,5,pass,foot_right,Khadija Monifa Shaw,Manchester City WFC
659,3852832,5820c1cc-7faa-4d1c-8e5e-aede657862cb,659,1,974.0,746,32210.0,32.382353,40.713924,-5.647059,0.516456,0,5,pass,foot_right,Laia Aleixandri,Manchester City WFC
2280,3852832,672fb667-c72b-4ef7-9413-0c9543b2c82e,2280,2,1255.5,968,10404.0,99.441176,35.635443,0.000000,0.000000,10,0,interception,foot,Noelle Maritz,Arsenal WFC


## Features & labels <a id="features"></a>

In [8]:
# Configure file and folder names
spadl_h5 = os.path.join(datafolder, "atomic-spadl-statsbomb.h5")
features_h5 = os.path.join(datafolder, "atomic-features.h5")
labels_h5 = os.path.join(datafolder, "atomic-labels.h5")
predictions_h5 = os.path.join(datafolder, "atomic-predictions.h5")

In [9]:
# Computes features
xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    fs.bodypart,
    fs.bodypart_onehot,
    fs.goalscore,
    fs.location,
    fs.polar,
    fs.movement_polar,
    fs.direction,
    fs.team,
    fs.time,
    fs.time_delta,
]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(features_h5) as featurestore:
    for game in tqdm.tqdm(list(df_games_city.itertuples()),desc=f"Generating and storing features in '{features_h5}'"):
        actions = spadlstore[f"atomic_actions/game_{game.game_id}"]

        # Convert actions to gamestate
        gamestates = fs.gamestates(atomicspadl.add_names(actions), 3) # A voir pourquoi pas 3 ?
        gamestates = fs.play_left_to_right(gamestates, game.home_team_id)

        X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
        featurestore[f"game_{game.game_id}"] = X

X.sample(2)

Generating and storing features in 'Data/data-spadl/atomic-features.h5': 100%|██████████| 6/6 [00:00<00:00,  8.88it/s]


,type_id_a0,type_id_a1,type_id_a2,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,type_interception_a0,type_shot_a0,type_shot_penalty_a0,type_shot_freekick_a0,type_keeper_save_a0,type_keeper_claim_a0,type_keeper_punch_a0,type_keeper_pick_up_a0,type_clearance_a0,type_bad_touch_a0,type_non_action_a0,type_dribble_a0,type_goalkick_a0,type_receival_a0,type_out_a0,type_offside_a0,type_goal_a0,type_owngoal_a0,type_yellow_card_a0,type_red_card_a0,type_corner_a0,type_freekick_a0,type_pass_a1,type_cross_a1,type_throw_in_a1,type_freekick_crossed_a1,type_freekick_short_a1,type_corner_crossed_a1,type_corner_short_a1,type_take_on_a1,type_foul_a1,type_tackle_a1,type_interception_a1,type_shot_a1,type_shot_penalty_a1,type_shot_freekick_a1,type_keeper_save_a1,type_keeper_claim_a1,type_keeper_punch_a1,type_keeper_pick_up_a1,type_clearance_a1,type_bad_touch_a1,type_non_action_a1,type_dribble_a1,type_goalkick_a1,type_receival_a1,type_out_a1,type_offside_a1,type_goal_a1,type_owngoal_a1,type_yellow_card_a1,type_red_card_a1,type_corner_a1,type_freekick_a1,type_pass_a2,type_cross_a2,type_throw_in_a2,type_freekick_crossed_a2,type_freekick_short_a2,type_corner_crossed_a2,type_corner_short_a2,type_take_on_a2,type_foul_a2,type_tackle_a2,type_interception_a2,type_shot_a2,type_shot_penalty_a2,type_shot_freekick_a2,type_keeper_save_a2,type_keeper_claim_a2,type_keeper_punch_a2,type_keeper_pick_up_a2,type_clearance_a2,type_bad_touch_a2,type_non_action_a2,type_dribble_a2,type_goalkick_a2,type_receival_a2,type_out_a2,type_offside_a2,type_goal_a2,type_owngoal_a2,type_yellow_card_a2,type_red_card_a2,type_corner_a2,type_freekick_a2,bodypart_id_a0,bodypart_id_a1,bodypart_id_a2,bodypart_foot_a0,bodypart_head_a0,bodypart_other_a0,bodypart_head/other_a0,bodypart_foot_a1,bodypart_head_a1,bodypart_other_a1,bodypart_head/other_a1,bodypart_foot_a2,bodypart_head_a2,bodypart_other_a2,bodypart_head/other_a2,goalscore_team,goalscore_opponent,goalscore_diff,x_a0,y_a0,x_a1,y_a1,x_a2,y_a2,dist_to_goal_a0,angle_to_goal_a0,dist_to_goal_a1,angle_to_goal_a1,dist_to_goal_a2,angle_to_goal_a2,mov_d_a0,mov_angle_a0,mov_d_a1,mov_angle_a1,mov_d_a2,mov_angle_a2,dx_a0,dy_a0,dx_a1,dy_a1,dx_a2,dy_a2,team_1,team_2,period_id_a0,time_seconds_a0,time_seconds_overall_a0,period_id_a1,time_seconds_a1,time_seconds_overall_a1,period_id_a2,time_seconds_a2,time_seconds_overall_a2,time_delta_1,time_delta_2
604,10,18,19,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0,0,0,True,False,False,False,True,False,False,False,True,False,False,False,0,1,-1,71.823529,30.643038,91.500000,37.615190,88.058824,30.987342,33.345875,0.100842,13.975679,0.261653,17.206963,0.175991,0.000000,0.000000,20.87521,-2.801057,0.000000,0.000000,-0.000000,-0.000000,-0.942576,-0.333992,-0.000000,-0.000000,False,True,1,901.0,901.0,1,900.0,900.0,1,899.0,899.0,1.0,2.0
1382,21,23,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [10]:
# Compute Labels 
yfns = [lab.scores, lab.concedes]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(labels_h5) as labelstore:
    for game in tqdm.tqdm(list(df_games_city.itertuples()), desc=f"Computing and storing labels in {labels_h5}"):
        actions = spadlstore[f"atomic_actions/game_{game.game_id}"]
        Y = pd.concat([fn(atomicspadl.add_names(actions)) for fn in yfns], axis=1)
        labelstore[f"game_{game.game_id}"] = Y

Y.sample(2)

Computing and storing labels in Data/data-spadl/atomic-labels.h5: 100%|██████████| 6/6 [00:00<00:00, 11.60it/s]


,scores,concedes
2157,False,False
367,False,False


## Computing the VAEP values <a id="vaep"></a>

In [11]:
# Recovering the model trained previously
pkl_filename =  os.path.join("modeles/", "mixed_VAEP_model.pkl")

with open(pkl_filename, 'rb') as file:
    models = pickle.load(file)

In [12]:
# Listing the features
nb_previous_actions = 3 

list_features = [
    #fs.actiontype,
    fs.actiontype_onehot,
    #fs.bodypart,
    fs.bodypart_onehot,
    fs.goalscore,
    fs.location,
    fs.polar,
    fs.movement_polar,
    fs.direction,
    fs.team,
    fs.time,
    fs.time_delta,
]

Xcols = fs.feature_column_names(list_features, nb_prev_actions=nb_previous_actions)
Ycols = ["scores", "concedes"]

In [13]:
# Creating the X matrix with the features for every game
X = []
for game_id in tqdm.tqdm(df_games_city.game_id, desc="Selecting features"):
    Xi = pd.read_hdf(features_h5, f"game_{game_id}")
    X.append(Xi[Xcols])
X = pd.concat(X).reset_index(drop=True)
X.sample(2)

Selecting features: 100%|██████████| 6/6 [00:00<00:00, 70.27it/s]


,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,type_interception_a0,type_shot_a0,type_shot_penalty_a0,type_shot_freekick_a0,type_keeper_save_a0,type_keeper_claim_a0,type_keeper_punch_a0,type_keeper_pick_up_a0,type_clearance_a0,type_bad_touch_a0,type_non_action_a0,type_dribble_a0,type_goalkick_a0,type_receival_a0,type_out_a0,type_offside_a0,type_goal_a0,type_owngoal_a0,type_yellow_card_a0,type_red_card_a0,type_corner_a0,type_freekick_a0,type_pass_a1,type_cross_a1,type_throw_in_a1,type_freekick_crossed_a1,type_freekick_short_a1,type_corner_crossed_a1,type_corner_short_a1,type_take_on_a1,type_foul_a1,type_tackle_a1,type_interception_a1,type_shot_a1,type_shot_penalty_a1,type_shot_freekick_a1,type_keeper_save_a1,type_keeper_claim_a1,type_keeper_punch_a1,type_keeper_pick_up_a1,type_clearance_a1,type_bad_touch_a1,type_non_action_a1,type_dribble_a1,type_goalkick_a1,type_receival_a1,type_out_a1,type_offside_a1,type_goal_a1,type_owngoal_a1,type_yellow_card_a1,type_red_card_a1,type_corner_a1,type_freekick_a1,type_pass_a2,type_cross_a2,type_throw_in_a2,type_freekick_crossed_a2,type_freekick_short_a2,type_corner_crossed_a2,type_corner_short_a2,type_take_on_a2,type_foul_a2,type_tackle_a2,type_interception_a2,type_shot_a2,type_shot_penalty_a2,type_shot_freekick_a2,type_keeper_save_a2,type_keeper_claim_a2,type_keeper_punch_a2,type_keeper_pick_up_a2,type_clearance_a2,type_bad_touch_a2,type_non_action_a2,type_dribble_a2,type_goalkick_a2,type_receival_a2,type_out_a2,type_offside_a2,type_goal_a2,type_owngoal_a2,type_yellow_card_a2,type_red_card_a2,type_corner_a2,type_freekick_a2,bodypart_foot_a0,bodypart_head_a0,bodypart_other_a0,bodypart_head/other_a0,bodypart_foot_a1,bodypart_head_a1,bodypart_other_a1,bodypart_head/other_a1,bodypart_foot_a2,bodypart_head_a2,bodypart_other_a2,bodypart_head/other_a2,goalscore_team,goalscore_opponent,goalscore_diff,x_a0,y_a0,x_a1,y_a1,x_a2,y_a2,dist_to_goal_a0,angle_to_goal_a0,dist_to_goal_a1,angle_to_goal_a1,dist_to_goal_a2,angle_to_goal_a2,mov_d_a0,mov_angle_a0,mov_d_a1,mov_angle_a1,mov_d_a2,mov_angle_a2,dx_a0,dy_a0,dx_a1,dy_a1,dx_a2,dy_a2,team_1,team_2,period_id_a0,time_seconds_a0,time_seconds_overall_a0,period_id_a1,time_seconds_a1,time_seconds_overall_a1,period_id_a2,time_seconds_a2,time_seconds_overall_a2,time_delta_1,time_delta_2
10057,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,3,0,3,21.529412,40.025316,14.294118,39.853165,14.294118,39.853165,83.687774,0.07206,90.894536,0.06444,90.894536,0.064440,19.072825,-1.26051,7.237342,0.023789,0.000000,0.000000,0.305331,-0.952246,0.999717,0.023787,0.000000,0.000000,True,True,1,1927.0,1927.0,1,1925.0,1925.0,1,1924.0,1924.0,2.0,3.0
12757,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,F

In [14]:
# Computing the vaep values
Y_pred = pd.DataFrame()
for col in ["scores", "concedes"]:
    Y_pred[col] = [p[1] for p in models[col].predict_proba(X)]

Y_pred.sample(2)

,scores,concedes
6047,0.001686,0.000304
3558,0.000898,0.000480


In [15]:
# get rows with game id per action
A = []
for game_id in tqdm.tqdm(games.game_id, "Loading game ids"):
    Ai = pd.read_hdf(spadl_h5, f"atomic_actions/game_{game_id}")
    A.append(Ai[["game_id"]])
A = pd.concat(A)
A = A.reset_index(drop=True)

Loading game ids: 100%|██████████| 6/6 [00:00<00:00, 69.13it/s]


In [16]:
# concatenate action game id rows with predictions and save per game
df_complete = pd.concat([A, Y_pred], axis=1)
grouped_predictions = df_complete.groupby("game_id")
for k, df in tqdm.tqdm(grouped_predictions, desc="Saving predictions per game"):
    df = df.reset_index(drop=True)
    df[Y_pred.columns].to_hdf(predictions_h5, f"game_{int(k)}")

Saving predictions per game: 100%|██████████| 6/6 [00:00<00:00, 62.27it/s]


In [17]:
#Compute VAEP values per action
df_actions_vaep = []
for game in tqdm.tqdm(list(df_games_city.itertuples()), desc="Loading actions"):
    actions = pd.read_hdf(spadl_h5, f"atomic_actions/game_{game.game_id}")
    actions = (
        atomicspadl.add_names(actions)
        .merge(players, how="left")
        .merge(teams, how="left",)
        .sort_values(["game_id", "period_id", "action_id"])
        .reset_index(drop=True)
    )
    preds = pd.read_hdf(predictions_h5, f"game_{game.game_id}")
    values = vaepformula.value(actions, preds.scores, preds.concedes)
    df_actions_vaep.append(pd.concat([actions, preds, values], axis=1))
df_actions_vaep = pd.concat(df_actions_vaep).sort_values(["game_id", "period_id", "time_seconds"]).reset_index(drop=True)
df_actions_vaep


Loading actions: 100%|██████████| 6/6 [00:00<00:00, 17.55it/s]


,game_id,original_event_id,action_id,period_id,time_seconds,team_id,player_id,x,y,dx,dy,type_id,bodypart_id,type_name,bodypart_name,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played,team_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,3852832,edbc9819-3d24-4c37-adc0-06949261903b,0,1,0.0,746,25554.0,52.058824,34.430380,-13.764706,3.184810,0,5,pass,foot_right,Khadija Monifa Shaw,None,21,True,23,Center Forward,100,Manchester City WFC,0.000522,0.000555,0.000000,-0.000000,0.000000
1,3852832,edbc9819-3d24-4c37-adc0-06949261903b,1,1,0.5,746,25632.0,38.294118,37.615190,0.000000,0.000000,23,0,receival,foot,Yui Hasegawa,None,25,True,10,Center Defensive Midfield,100,Manchester City WFC,0.000936,0.000758,0.000414,-0.000203,0.000211
2,3852832,ce398ea2-769b-43d0-8911-a71d6ad7ce82,2,1,1.0,746,25632.0,38.294118,37.615190,1.235294,0.172152,21,0,dribble,foot,Yui Hasegawa,None,25,True,10,Center Defensive Midfield,100,Manchester City WFC,0.001365,0.000901,0.000429,-0.000143,0.000286
3,3852832,9fdba1c7-4244-43fd-b0fc-b6f944407c1f,3,1,2.0,746,25632.0,39.529412,37.787342,-12.617647,-16.956962,0,5,pass,foot_right,Yui Hasegawa,None,25,True,10,Center Defensive Midfield,100,Manchester City WFC,0.000629,0.000602,-0.000736,0.000299,-0.000437
4,3852832,9fdba1c7-4244-43fd-b0fc-b6f944407c1f,4,1,2.5,746,10185.0,26.911765,20.830380,0.000000,0.000000,23,0,receival,foot,Stephanie Houghton,None,6,True,3,Right Center Back,100,Manchester City WFC,0.000753,0.000589,0.000124,0.000013,0.000137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18513,3856040,a83836bd-cded-4d49-8156-6da7b6f4298d,3256,2,3040.0,746,4649.0,5.911765,18.420253,1.941176,0.000000,21,0,dribble,foot,Esme Beth Morgan,Esme Morgan,14,False,0,Substitute,12,Manchester City WFC,0.002152,0.001568,0.001254,0.000344,0.001597
18514,3856040,c33b1ed4-5f74-4058-b0cc-140feaba7c06,3257,2,3040.0,746,4649.0,7.852941,18.420253,-4.323529,11.275949,0,5,pass,foot_right,Esme Beth Morgan,Esme Morgan,14,False,0,Substitute,12,Manchester City WFC,0.000763,0.002578,-0.001389,-0.001010,-0.002399
18515,3856040,c33b1ed4-5f74-4058-b0cc-140feaba7c06,3258,2,3041.0,746,4637.0,3.529412,29.696203,0.000000,0.000000,23,0,receival,foot,Ellie Roebuck,None,1,True,1,Goalkeeper,97,Manchester City WFC,0.001252,0.004574,0.000490,-0.001996,-0.001506
18516,3856040,52778e78-722d-44c4-99f4-8918fee40eec,3259,2,3042.0,746,4637.0,3.529412,29.696203,0.000000,0.000000,21,0,dribble,foot,Ellie Roebuck,None,1,True,1,Goalkeeper,97,Manchester City WFC,0.000825,0.005466,-0.000428,-0.000892,-0.001320


## Use the position to specialized the VAEP <a id="position"></a>

In [18]:
df_actions_vaep[df_actions_vaep.type_name == 'goal']["offensive_value"].describe()

count    19.000000
mean      0.680180
std       0.127933
min       0.466698
25%       0.614366
50%       0.650713
75%       0.776564
max       0.878507
Name: offensive_value, dtype: float64

In [19]:
df_actions_vaep[df_actions_vaep.type_name == 'shot']["offensive_value"].describe()

count    175.000000
mean       0.065208
std        0.073440
min       -0.093760
25%        0.023568
50%        0.043097
75%        0.093191
max        0.479457
Name: offensive_value, dtype: float64

The VAEP for a goal is really high compared to the other events. As we want to analyze the game of the player, we do not want a goal to make the VAEP skyrocks during a game. We are going to divide the offensive value

In [20]:
df_actions_vaep["offensive_value"] = df_actions_vaep.apply(
    lambda x : x["offensive_value"]/10 if x["type_name"] == 'goal' else x["offensive_value"], axis=1) 

We are going to analyze the game with the real time played - so we do not have two '46mn', 1 for the extra-time in the 1st period and the other one at the beginning at the 2nd period. We are computing a "real time in mn" for this purpose

In [21]:
# For every game, use a real time for every minute played
list_to_concat = []
for game_id in df_actions_vaep["game_id"].unique():
    df_game = df_actions_vaep[df_actions_vaep["game_id"] == game_id]
    df_game["real_time_mn"] = df_game.apply(lambda x : round(x["time_seconds"]/60, 3) if x["period_id"] == 1 else None, axis=1) 
    max_time_first_period = df_game["real_time_mn"].max()
    df_game["real_time_mn"] = df_game.apply(lambda x : round(x["time_seconds"]/60 + max_time_first_period, 3) if x["period_id"] == 2 else x["real_time_mn"], axis=1) 
    df_game["round_real_time_mn"] = df_game["real_time_mn"].apply(ceil)
    list_to_concat.append(df_game)

df_actions_vaep = pd.concat(list_to_concat)

In [22]:
# Add the positions by event
list_to_concat = []
for game_id in df_actions_vaep["game_id"].unique():
    df_events = SBL.events(game_id=game_id)
    list_to_concat.append(df_events)

df_events = pd.concat(list_to_concat)

df_actions_vaep = df_actions_vaep.merge(df_events[["game_id","event_id", "player_id","position_name"]].drop_duplicates(),
            how='left', left_on=["game_id",'original_event_id',"player_id"], right_on=['game_id',"event_id", "player_id"])


In [23]:
# If the position name is not specified we take the starting position name
df_actions_vaep.loc[
    df_actions_vaep["position_name"].isna(), "position_name"] = df_actions_vaep.loc[
    df_actions_vaep["position_name"].isna(), "starting_position_name"]

# If there is no position name and the starting position name is substitute, do not specify a position name
df_actions_vaep.loc[df_actions_vaep["position_name"] == "Substitute", "position_name"] = np.NaN

# If there is still no position, look at the common position of the player
list_missing_position = df_actions_vaep[df_actions_vaep["position_name"].isna()]["player_id"].unique()
missing = df_events[["player_id", "position_name"]].drop_duplicates()
misssing = missing[missing["player_id"].isin(list_missing_position)]
missing = misssing.drop_duplicates(subset='player_id', keep=False)
dict_missing = missing.set_index("player_id").to_dict("index")

df_actions_vaep["position_name"] = df_actions_vaep.apply(
    lambda x : dict_missing[x["player_id"]]["position_name"]
    if (pd.isnull(x["position_name"]) and x["player_id"] in dict_missing.keys())
    else x["position_name"], axis=1)


In [24]:
# Get the matrix computed previously
matrix = pd.read_csv("matrice_vaep.csv")
matrix = matrix.drop(index=0).drop(columns="Unnamed: 0")

matrix.rename(columns={
    "offensive_value" : "offensive_value_mean",
    "offensive_value.1" : "offensive_value_std",
    "defensive_value" : "defensive_value_mean",
    "defensive_value.1" : "defensive_value_std",
}, inplace=True)

matrix = matrix.astype(
    {
    "offensive_value_mean" : float,
    "offensive_value_std" : float,
    "defensive_value_mean" : float,
    "defensive_value_std" : float}
)
matrix.head()

,type_name,poste_type,offensive_value_mean,offensive_value_std,defensive_value_mean,defensive_value_std
1,bad_touch,Defender,-0.008133,0.020782,-0.013119,0.084050
2,bad_touch,Forward,-0.012378,0.023891,-0.001613,0.019895
3,bad_touch,Goalkeeper,-0.000240,0.000176,-0.469585,0.346637
4,bad_touch,Midfield,-0.008805,0.018521,-0.002241,0.016161
5,clearance,Defender,0.000505,0.001339,0.001565,0.026696


In [25]:
# Weight the offensive and defensive values regarding the position of the name
value_poste = pd.read_csv("poste.csv", sep=";").set_index("position_name").to_dict("index")
value_poste["Center Back"]

{'type': 'Defender', 'offensive': 0.2, 'defensive': 0.8}

In [26]:
df_actions_vaep["poste_type"] = df_actions_vaep["position_name"].apply(lambda x : value_poste[x]["type"] if not pd.isnull(x) else None)
df_actions_vaep[["game_id", "player_name", "position_name", "poste_type"]].sample(3)

,game_id,player_name,position_name,poste_type
7645,3855961,Laura Coombs,Left Center Midfield,Midfield
10445,3855980,Katie Robinson,Right Midfield,Midfield
1008,3852832,Laia Aleixandri López,Left Back,Defender


In [27]:
# Add the distribution information to the dataframe
df_actions_vaep = df_actions_vaep.merge(matrix, how='left', on=["poste_type", "type_name"])

In [28]:
# Adjust the offensive and defensive values of the events regarding the distribution
df_actions_vaep["adjusted_offensive"] = (
    (df_actions_vaep["offensive_value"] - df_actions_vaep["offensive_value_mean"]) / (df_actions_vaep["offensive_value_std"]))
df_actions_vaep["adjusted_defensive"] = (
    (df_actions_vaep["defensive_value"] - df_actions_vaep["defensive_value_mean"]) / (df_actions_vaep["defensive_value_std"]))

In [29]:
# Adjust the offensive and defensive values of the events regarding the roles they have
df_actions_vaep["offensive_part"] = df_actions_vaep["position_name"].apply(
                lambda x : value_poste[x]["offensive"] if not pd.isnull(x) else None)
df_actions_vaep["defensive_part"] = df_actions_vaep["position_name"].apply(
                lambda x : value_poste[x]["defensive"] if not pd.isnull(x) else None)

In [30]:
# Create an adjusted VAEP 
df_actions_vaep["adjusted_vaep"] = df_actions_vaep.apply(lambda x : 
                    ((x["offensive_part"] * x["adjusted_offensive"]) + (x["defensive_part"] * x["adjusted_defensive"])) , axis=1)

## Analysing a game <a id="analyse"></a>

In [31]:
# Divinding the data between the preivous games and the arsenal game
df_previous_games = df_actions_vaep[df_actions_vaep["game_id"] != game_arsenal_id]
df_arsenal_game = df_actions_vaep[df_actions_vaep["game_id"] == game_arsenal_id]

In [32]:
a = df_arsenal_game[df_arsenal_game.period_id == 1].iloc[-1]["real_time_mn"]
a

51.067

In [33]:
def compute_cumulative_vaep(df_game:pd.DataFrame) -> pd.DataFrame:
    """
    Compute the cumulative adjusted vaep of a game

    Arg: df_game (pd.DataFrame)
    Returns: pd.DataFrame: 
    """
    
    list_calcul_by_mn = []
    tmp_final_match = df_game["real_time_mn"].iloc[-1]

    for time in range(3, ceil(tmp_final_match)+1, 1):

        end_of_1st = df_game[df_game.period_id == 1].iloc[-1]["real_time_mn"]

        df_game_time = df_game[df_game["real_time_mn"] < time]
    
        # Compute each player's number of actions and total VAEP values
        playersR = (
            df_game_time[["game_id", "player_id","jersey_number", "team_name","adjusted_vaep"]]
            .groupby(["player_id", "jersey_number","team_name","game_id"])
            .sum()
            .reset_index()
        )
        # Add player names
        playersR = playersR.merge(players[players.game_id == game_arsenal_id]
                                [["player_id", "player_name", "is_starter","minutes_played"]].drop_duplicates()
                                , how="left")
        
        # Work on a visual clock time
        period = df_game_time["period_id"].iloc[-1]

        playersR["time"] = time
        playersR["period"] = period
        
        list_calcul_by_mn.append(playersR)

    calcul_by_mn = pd.concat(list_calcul_by_mn)

    # Change time format
    calcul_by_mn['official_clock'] = np.where(calcul_by_mn['period'] == 1,
                        np.where(calcul_by_mn['time'] > 45,
                                calcul_by_mn['time'].apply(lambda x: str(45) + "'+" + str(x - 45)),
                                calcul_by_mn['time'].astype(str)
                        ), 
                        np.where(calcul_by_mn['time'] > 45 + end_of_1st,
                                calcul_by_mn['time'].apply(lambda x: str(90) + "'+" + str(round(x - end_of_1st-45))),
                                calcul_by_mn['time'].apply(lambda x: str(round(x-end_of_1st)+45))
                        ),
                        )
    return calcul_by_mn

The 5 previous games

In [34]:
list_to_concat = []

for game_id in df_previous_games.game_id.unique():
    df_game = df_previous_games[df_previous_games.game_id == game_id]
    df = compute_cumulative_vaep(df_game)
    list_to_concat.append(df)

df = pd.concat(list_to_concat)
df.sample(3)

,player_id,jersey_number,team_name,game_id,adjusted_vaep,player_name,is_starter,minutes_played,time,period,official_clock
17,32210.0,4,Manchester City WFC,3855961,-6.177066,Laia Aleixandri López,True,100.0,28,1,28
7,15421.0,10,Aston Villa,3856030,-0.885153,NaN,NaN,NaN,6,1,6
1,4649.0,14,Manchester City WFC,3855947,-1.309512,NaN,NaN,NaN,5,1,5


In [35]:
# Analyzing mn par mn what the players did
df_previous_games_grouped = df.groupby(["player_name","player_id","team_name","time"]).mean()[["adjusted_vaep"]]
df_previous_games_grouped = df_previous_games_grouped.reset_index()
df_previous_games_grouped

,player_name,player_id,team_name,time,adjusted_vaep
0,Alex Greenwood,10252.0,Manchester City WFC,3,-0.278111
1,Alex Greenwood,10252.0,Manchester City WFC,4,-0.441717
2,Alex Greenwood,10252.0,Manchester City WFC,5,-0.703733
3,Alex Greenwood,10252.0,Manchester City WFC,6,-0.726911
4,Alex Greenwood,10252.0,Manchester City WFC,7,-1.184834
...,...,...,...,...,...
1110,Yui Hasegawa,25632.0,Manchester City WFC,96,-11.347006
1111,Yui Hasegawa,25632.0,Manchester City WFC,97,-11.254228
1112,Yui Hasegawa,25632.0,Manchester City WFC,98,-15.281961
1113,Yui Hasegawa,25632.0,Manchester City WFC,99,-14.579383


In [36]:
df_previous_games_grouped.to_csv("export/other_games_grouped_cumulative_vaep.csv", index=False)

Arsenal game -  Computed the cumulative VAEP per minute played

In [37]:
arsenal_data = compute_cumulative_vaep(df_arsenal_game)

In [38]:
arsenal_data.to_csv("export/arsenal_game_cumulative_vaep.csv", index=False)

Gather data in one graph

In [39]:
def trace_player(player:str, df_games:pd.DataFrame, df_previous_games_grouped:pd.DataFrame):
    df_previous = df_previous_games_grouped[df_previous_games_grouped["player_name"] == player]
    df_game = df_games[df_games["player_name"] == player]

    fig = go.Figure(
        data =[
            {'x': df_game.time, 'y': df_game.adjusted_vaep, 'name' : "Arsenal Game"},
            {'x': df_previous.time, 'y': df_previous.adjusted_vaep, 'name' : "Mean"}
            ],
        layout=go.Layout(
            title=go.layout.Title(text=player)))
    fig.show()

In [40]:
trace_player("Chloe Kelly", arsenal_data, df_previous_games_grouped)

In [41]:
trace_player("Lauren Hemp", arsenal_data, df_previous_games_grouped)

In [42]:
trace_player("Stephanie Houghton", arsenal_data, df_previous_games_grouped)

In [43]:
trace_player("Leah Williamson", arsenal_data, df_previous_games_grouped)

## Provinding substitute ideas <a id="substitute"></a>

List the played positions of each player

In [44]:
list_to_concat = []

for game_id in df_games_city.game_id.unique():
    with open('Data/StatsBomb-City-VAEP/Data/lineups/{}.json'.format(game_id)) as f:
        lineups = json.load(f)

    _, df_lineups,_ = extract_dataframe_from_json(lineups)
    list_to_concat.append(df_lineups[df_lineups["team_name"] == 'Manchester City WFC'][["player_name","player_id", "position"]])

df_positions = pd.concat(list_to_concat).drop_duplicates().sort_values(by="player_name")
df_positions.sample(3)

,player_name,player_id,position
1,Esme Beth Morgan,4649,Left Back
12,Ruby Mace,41812,Center Defensive Midfield
16,Julie Blakstad,191851,Left Wing


In [45]:
# Sum the minuted played and the vaep on these games
df_ranking = df_previous_games[["player_id", "minutes_played", "adjusted_vaep"]].groupby("player_id").sum().reset_index()
df_ranking["vaep_rating_per_mn_played"] = df_ranking.adjusted_vaep * 90 / df_ranking.minutes_played
df_ranking

,player_id,minutes_played,adjusted_vaep,vaep_rating_per_mn_played
0,4637.0,50999,-62.613322,-0.110496
1,4638.0,9010,10.365701,0.103542
2,4649.0,55581,-12.895195,-0.020881
3,4651.0,42,-0.313043,-0.670806
4,4848.0,11640,14.762777,0.114145
...,...,...,...,...
85,135758.0,598,-1.237244,-0.186207
86,135759.0,5346,-2.910502,-0.048998
87,191851.0,15368,-7.056131,-0.041323
88,221888.0,56179,-24.846589,-0.039805


In [46]:
# Merge with the positions
df_substitutes_proposition = df_positions.merge(df_ranking[["player_id", "vaep_rating_per_mn_played"]]).sort_values(by=["position", "vaep_rating_per_mn_played"], ascending=[True, False])
df_substitutes_proposition[df_substitutes_proposition.position == "Left Back"]

,player_name,player_id,position,vaep_rating_per_mn_played
9,Esme Beth Morgan,4649,Left Back,-0.020881
22,Laia Aleixandri López,32210,Left Back,-0.062545
27,Leila Ouahabi El Ouahabi,10163,Left Back,-0.104180
5,Demi Stokes,4651,Left Back,-0.670806


In [47]:
list_starting_players = players[
    (players.game_id == game_arsenal_id) & 
    (players.team_id == 746) & 
    (players.starting_position_name != "Substitute")]["player_id"]

In [48]:
df_substitutes_proposition = df_substitutes_proposition[
    ~df_substitutes_proposition["player_id"].isin(list_starting_players)
    ]

In [49]:
df_substitutes_proposition.to_csv("export/vaep_rating_on_previous_games_per_positions.csv", index=False)